In [1]:
import sys

sys.path.append("..")

from lakeml.vaex import Pipeline
models_path = '../models'
import lakefs_client
from lakefs_client.models import RepositoryCreation
from lakefs_client.client import LakeFSClient
from dotenv import load_dotenv
import os
import vaex
import sys
from collections import defaultdict
from lakeml.vaex.core.variables import *
import warnings
warnings.filterwarnings('ignore')

sys.path.append("..")
from lakeml.vaex import Pipeline
models_path = '../models'

load_dotenv()


configuration = lakefs_client.Configuration()
configuration.username = os.getenv('LAKEFS_USER')
configuration.password = os.getenv('LAKEFS_PASSWORD')
configuration.host = os.getenv('LAKEFS_HOST')
client= LakeFSClient(configuration)

def lakefs_open(path):
    splits = path.split('/')
    repo = splits[2]
    branch = splits[3]
    filename = splits[4]
    path = f"../data/{repo}/{branch}/{filename}"
    ret = vaex.open(path)
    ret.variables['lakefs'] = {'repo':repo,'branch':branch}
    return ret

def lakefs_commit(pipeline, path, hooks=None):
    splits = path.split('/')
    repo = splits[2]
    branch = splits[3]
    filename = splits[4]
    path = f"../data/{repo}/{branch}/{filename}"
    if hooks is not None:
        for hook in hooks:
            if hook(pipeline) is False:
                raise RuntimeError("invalid pipeline")
    pipeline.save(path)

def lakefs_load(path):
    splits = path.split('/')
    repo = splits[2]
    branch = splits[3]
    filename = splits[4]
    path = f"../data/{repo}/{branch}/{filename}"
    return Pipeline.from_file(path)

def validate_evaluation(pipeline):
    print('validateing "accuracy" ')
    safe = True
    for key in ['accuracy']:
        if key not in pipeline.variables.get('evaluation',{}):
            print(f"{key} is missing")
            safe = False
    return safe

def validate_output(pipeline):
    print('validateing "prediction" column exists')
    if not 'prediction' in pipeline.example.column_names:
        print("prediction column is missing")
    return 'prediction' in pipeline.example.column_names        


@vaex.register_dataframe_accessor('lakefs', override=True)
class LakeFSAccessor(object):
    def __init__(self, df):
        self.df = df
        self.client = LakeFSClient(configuration)
        
    def _list_branches(self, repo=None):
        repo = repo or self.df.lakefs.repo
        return self.client.branches.list_branches(repo)

    @property
    def branch(self):
        return self.df.variables.get('lakefs',{}).get('branch')
    
    @property
    def branches(self):
        return self._list_branches()['results']
    
    @property
    def repo(self):
        return self.df.variables.get('lakefs',{}).get('repo')
# client.repositories.create_repository(RepositoryCreation(name='server', storage_namespace='local:///Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/data/server', default_branch='main'))

In [2]:
# server a (classification/regression )
pipeline = lakefs_load('lakefs://server/lightgbm/pipeline.pkl')
pipeline.inference(pipeline.example)

#,sepal_length,sepal_width,petal_length,petal_width,class_,lgm_predictions,prediction
0,5.9,3,4.2,1.5,1,"'array([6.04909665e-09, 9.99999993e-01, 1.266636...",1


In [4]:
# server b (nearest neighbours)
pipeline = lakefs_load('lakefs://server/knn/pipeline.pkl')
pipeline.inference(pipeline.example)

#,id,x,y,z,vx,vy,vz,E,L,Lz,FeH,knn
0,0,1.23187,-0.396929,-0.598058,301.155,174.059,27.4275,-149431,407.389,333.956,-1.00539,"array([13, 0, 21], dtype=uint64)"


In [2]:
# server c - (recommendations)
pipeline = lakefs_load('lakefs://server/recommender/pipeline.pkl')
pipeline.inference(pipeline.example)

ERROR:MainThread:vaex:error evaluating: name at rows 0-1
Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 106, in evaluate
    result = self[expression]
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 166, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: "Unknown variables or column: 'title'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/dataframe.py", line 2034, in data_type
    data = self.evaluate(expression, 0, 1, filtered=False, array_type=array_type, parallel=False)
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-pa

ERROR:vaex:error evaluating: name at rows 0-1
Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 106, in evaluate
    result = self[expression]
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 166, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: "Unknown variables or column: 'title'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/dataframe.py", line 2034, in data_type
    data = self.evaluate(expression, 0, 1, filtered=False, array_type=array_type, parallel=False)
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex

ERROR:MainThread:vaex:error evaluating: title at rows 0-1
Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 106, in evaluate
    result = self[expression]
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 166, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: "Unknown variables or column: 'title'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/dataframe.py", line 2034, in data_type
    data = self.evaluate(expression, 0, 1, filtered=False, array_type=array_type, parallel=False)
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-p

ERROR:vaex:error evaluating: title at rows 0-1
Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 106, in evaluate
    result = self[expression]
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 166, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: "Unknown variables or column: 'title'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/dataframe.py", line 2034, in data_type
    data = self.evaluate(expression, 0, 1, filtered=False, array_type=array_type, parallel=False)
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vae

ERROR:MainThread:vaex:error evaluating: url at rows 0-1
Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 106, in evaluate
    result = self[expression]
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 166, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: "Unknown variables or column: 'title'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/dataframe.py", line 2034, in data_type
    data = self.evaluate(expression, 0, 1, filtered=False, array_type=array_type, parallel=False)
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-pac

...

ERROR:MainThread:vaex:error evaluating: url at rows 0-1
Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 106, in evaluate
    result = self[expression]
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 166, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: "Unknown variables or column: 'title'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/dataframe.py", line 2034, in data_type
    data = self.evaluate(expression, 0, 1, filtered=False, array_type=array_type, parallel=False)
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-pac

ERROR:vaex:error evaluating: url at rows 0-1
Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 106, in evaluate
    result = self[expression]
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 166, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: "Unknown variables or column: 'title'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/dataframe.py", line 2034, in data_type
    data = self.evaluate(expression, 0, 1, filtered=False, array_type=array_type, parallel=False)
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/

ERROR:MainThread:vaex:error evaluating: genres at rows 0-1
Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 106, in evaluate
    result = self[expression]
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 166, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: "Unknown variables or column: 'title'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/dataframe.py", line 2034, in data_type
    data = self.evaluate(expression, 0, 1, filtered=False, array_type=array_type, parallel=False)
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-

ERROR:vaex:error evaluating: genres at rows 0-1
Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 106, in evaluate
    result = self[expression]
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/scopes.py", line 166, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: "Unknown variables or column: 'title'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/vaex/dataframe.py", line 2034, in data_type
    data = self.evaluate(expression, 0, 1, filtered=False, array_type=array_type, parallel=False)
  File "/Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/venv/lib/python3.7/site-packages/va

#,userId,movieId,rating,name,title,year,url,genres,als,tfidf,explanation
0,156,2,5,error,error,1995,error,error,"""['Presidio, The', 'Blind Date', 'Greystoke: The...","""['Star Wars: Episode IV - A New Hope', 'Indiana...","""['Exit to Eden', 'Renaissance Man', 'Man Withou..."


In [10]:
# server d (NLP - spacy)
from goldilocks.vaex import Pipeline
pipeline = Pipeline.from_file(f"{models_path}/spacy.pkl") 
pipeline.inference(pipeline.example)

#,text,organisations,money,vector
0,"""'Net income was $9.4 million compared to the pr...",['Apple'],['$9.4 million' '$2.7 million'],'[ 0.46828035 0.30820569 0.40490758 -0.1441493...


In [11]:
# and so on... (NLP - hagging face)
from goldilocks.vaex import Pipeline
pipeline = Pipeline.from_file(f"{models_path}/hf.pkl") 
pipeline.inference(pipeline.example)

#,text,results,label,score
0,"""'We are very happy to include pipeline into the...","{'label': 'POSITIVE', 'score': 0.9978193640708923}",'POSITIVE',0.997819


### Extra features

In [12]:
import json 
pipeline = Pipeline.from_file('../models/lgm.pkl')
print(f"Missing values or columns?: {{'sepal_length':5,'petal_width':1}}")
print(json.dumps(pipeline.inference({'sepal_length':5,'petal_width':1}).gl.to_records(0), indent=4, sort_keys=True))

Missing values or columns?: {'sepal_length':5,'petal_width':1}
{
    "class_": 0,
    "lgm_predictions": [
        0.9999999689996125,
        2.1232029335696277e-08,
        9.768358172292381e-09
    ],
    "petal_length": 0,
    "petal_width": 1,
    "prediction": 0,
    "sepal_length": 5,
    "sepal_width": 0
}


In [13]:
print(f"Need an example?:")
print(json.dumps(pipeline.example.gl.to_records(0), indent=4, sort_keys=True))

Need an example?:
{
    "class_": 1,
    "lgm_predictions": [
        6.0490966479191e-09,
        0.9999999926842671,
        1.2666360957863501e-09
    ],
    "petal_length": 4.2,
    "petal_width": 1.5,
    "prediction": 1,
    "sepal_length": 5.9,
    "sepal_width": 3.0
}


In [14]:
print(f"Forgot the original columns?: {pipeline.input_columns}")

Forgot the original columns?: ['petal_width', 'sepal_length', 'sepal_width', 'petal_length', 'class_']


In [15]:
print(f"Important varaibles used?: {pipeline.variables}")

Important varaibles used?: {'accuracy': 1.0}


## Refit

# One lambda to rule them all